# Youtube $\rightarrow$ iTunes
---

First, let's import some standard packages we might need

In [18]:
# misc
import os
import shutil
import math
import datetime
# plots
import matplotlib.pyplot as plt
%matplotlib inline


## Test Download Methods

### Method 1: Using PyTube

In [15]:
!pip install pytube

Note: you may need to restart the kernel to use updated packages.


In [16]:
from pytube import YouTube

In [17]:
!conda list | grep pytube

pytube                    11.0.1                   pypi_0    pypi


Download **video** with `pytube`

In [5]:
video_url = 'https://www.youtube.com/watch?v=YWhSQpUNGgY'
# video.streams.filter(file_extension = "mp4").all()
youtube = YouTube(video_url)
video = youtube.streams.first()
video.download(filename="Sticks_and_Stones.mp4")
# video.filesize

'/Users/colbyr/Documents/GitHub/downloader/Sticks_and_Stones.mp4'

Download *only* **audio** with `pytube`

In [13]:
youtube_video_url = 'https://youtu.be/bIjCNQT5BNU'
track_title = '8hr Binarual Sleep.mp3'
 
yt_obj = YouTube(youtube_video_url)

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
audio = yt_obj.streams.filter(only_audio=True).first()
audio.download('./downloads', filename=track_title)

'/Users/colbyr/Documents/GitHub/downloader/./downloads/8hr Binarual Sleep.mp3'

### Alternate Method (NOT USING)
This method employs the `requests` module, but will not be used because it does not seem to work consistently

In [ ]:
# import requests
# file_url = "https://www.youtube.com/watch?v=YWhSQpUNGgY"
# file_name = "Sticks and Stone.mp4"

# r = requests.get(file_url, stream = True)

# # download started 
# with open(file_name, 'wb') as f: 
#     for chunk in r.iter_content(chunk_size = 1024*1024): 
#         if chunk: 
#             f.write(chunk) 
  
# print( "%s downloaded!\n"%file_name )

Sticks and Stone.mp4 downloaded!



# Define Download Functions
---

Import modules again to start from this cell:

In [6]:
import os
from pytube import YouTube

## **Video** *Download* and *Conversion* Functions

Download function:
* download first video stream with `pytube`

In [23]:
def download_video(youtube_url, song_title, file_path='.'):
  video_filename = song_title + ".mp4"
  yt_obj = YouTube(youtube_url)
  video = yt_obj.streams.first()
  video.download(output_path=file_path, filename=video_filename)

Conversion Function:
* run `ffmpeg` with `os.system()` module

In [25]:
def convert_to_audio(song_title, file_path='.', output_type="mp3"):
    vid = f'{song_title}.mp4'
    audio = f'{song_title}.{output_type}'

    # set variable with path to video and audio files
    vid_path = os.path.join(file_path, vid)
    audio_path = os.path.join(file_path, audio)
    
    # call `ffmpeg` to transform to output format
    os.system(f"ffmpeg -i '{vid_path}' '{audio_path}'")

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## **Audio** Download Function
* Download audio using `.filter(only_audio)` option from `pytube`

In [26]:
def download_audio(youtube_url, song_title, file_path='.'):
  audio_filename = f"{song_title}.mp3"
  yt_obj = YouTube(youtube_url)
  audio = yt_obj.streams.filter(only_audio=True).first()
  audio.download(output_path=file_path, filename=audio_filename)

Test download function

In [37]:
youtube_video_url = "https://www.youtube.com/watch?v=YWhSQpUNGgY"

fname = "01 Sticks and Stones"

output_path = "./downloads"

download_audio(youtube_video_url, fname, output_path)

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Edit Metadata
---

## Edit ID3 Tags
Use either `mutagen` or `eyeD3`
* [mutagen](https://mutagen.readthedocs.io/en/latest/): `pip install mutagen`
* [eyeD3](https://eyed3.readthedocs.io/en/latest/): `pip install eyed3` 

### Check current metadata 
Use `ffprobe`, included in `ffmpeg` binary, to return the metadata currently attributed to the `.mp3` file

In [61]:
import subprocess
!ffprobe "./downloads/01 Sticks and Stones.mp3"

ffprobe version 4.3.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with clang version 11.0.0
  configuration: --prefix=/Users/colbyr/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1609681034781/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './downloads/01 Sticks an

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Install and Test `eyeD3`

In [ ]:
!pip install eyed3

In [87]:
import eyed3

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [97]:
path = "./downloads/01 Sticks and Stones.mp3"

audio_file = eyed3.mp3.Mp3AudioFile(path)
audio_file.initTag()
audio_file.tag.artist = "Kings Kaleidoscope"
audio_file.tag.album = "Beauty Between"
audio_file.tag.genre = "Alternative"
audio_file.tag.track_num = 1

audio_file.tag.save()

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [98]:
!ffprobe './downloads/01 Sticks and Stones.mp3'

ffprobe version 4.3.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with clang version 11.0.0
  configuration: --prefix=/Users/colbyr/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1609681034781/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f90f700ca00] Discarding ID3 tags be

In [120]:
af = eyed3.load(path)
af.tag.album = "Beauty Between"
af.tag.artist = "Kings Kaleidoscope" 

af.tag.save()

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [121]:
print(af.tag.title)

None


In [108]:
!ffprobe "./downloads/01 Sticks and Stones.mp3"

ffprobe version 4.3.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with clang version 11.0.0
  configuration: --prefix=/Users/colbyr/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1609681034781/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f9b2400be00] Discarding ID3 tags be

/Users/colbyr/miniconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Using `Mutagen`
* `eyed3` seems to be having problems and documentation is inconsistent with available version(s) 
* also seems like iTunes mainly uses `.m4a` or `.m4p` file type, which `mutagen` explicitly supports

$\rightarrow$ switching to trying out `mutagen`

### Helpful Notes for using `Mutagen`:
* The `ytmdl` repo ([here](https://github.com/deepjyoti30/ytmdl/tree/b016582520b9209a7481d7d8b18c8befd889e477)) is really helpful for referencing methods to add metadata (either manually or by searching Spotify/iTunes/etc)
* See [Mutagen docs](https://mutagen.readthedocs.io/en/latest/api/mp4.html#mutagen.mp4.MP4Tags) for `ID3 tag` keys

In [123]:
!pip install mutagen

Import `mutagen` modules

In [1]:
from mutagen.id3 import (
    ID3,
    APIC,
    TIT2,
    TPE1,
    TALB,
    TCON,
    TRCK,
    TYER,
    PictureType
)
from mutagen.mp3 import MP3
from mutagen.mp4 import MP4, MP4Cover
from mutagen import File

Set audio path and `MP4` object

In [34]:
song_path = "downloads/sleep.m4a"
audio = MP4(song_path)

Initialize tags if tags do not already exist

In [38]:
try:
    audio.add_tags()
except Exception as e:
    print(e)

audio.save()

an MP4 tag already exists


Use `ffprobe` to check metadata before editing with `mutagen`

In [39]:
!ffprobe "./downloads/sleep.m4a"

ffprobe version 4.3.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with clang version 11.0.0
  configuration: --prefix=/Users/colbyr/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1609681034781/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './downloads/sleep.m4a':


Set variables to be used in ID3 tag editing

In [40]:
track_name = "Sleep Music (Binaural Beats)"
collection_name = "Sleep"
artist_name = "GreenRed Productions"
primary_genre_name = "Binaural Beats"
release_date = "2018"

See [Mutagen docs](https://mutagen.readthedocs.io/en/latest/api/mp4.html#mutagen.mp4.MP4Tags) for `ID3 tag` keys

In [51]:
# tag keys here: https://mutagen.readthedocs.io/en/latest/api/mp4.html#mutagen.mp4.MP4Tags

audio["\xa9nam"] = track_name
audio["\xa9alb"] = collection_name
audio["\xa9ART"] = artist_name
audio["\xa9day"] = release_date
audio["\xa9gen"] = primary_genre_name

Save edited tags

In [42]:
audio.save()

Check metadata with `ffprobe` post-editing

In [44]:
!ffprobe "./downloads/sleep.m4a"

ffprobe version 4.3.1 Copyright (c) 2007-2020 the FFmpeg developers
  built with clang version 11.0.0
  configuration: --prefix=/Users/colbyr/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1609681034781/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './downloads/sleep.m4a':


# Export to iTunes/Apple-Music

## Test methods for accessing Music Library

### Connect to `.itl` or `.musiclibrary` file

### Edit and Re-Import `.xml` file from Music `export` function
Helpful articles:
* This [article](https://keystrokecountdown.com/articles/itunes/index.html) (note: written in `python2`, but principles should be transferrable)
    * makes use of `plistlib` module (docs [here](https://docs.python.org/3/library/plistlib.html))
* This [tutorial](https://junschoi.github.io/posts/itunes_library_analysis/) on analyzing iTunes data
    * utilizes `ElementTree` module from `xml.etree`

In [65]:
import plistlib
import xml.etree.ElementTree as ET
import pandas as pd

In [66]:
lib_path = "test_library/Library.xml"
tree = ET.parse(lib_path)
root = tree.getroot()

In [68]:
root_lst = []

for i,x in enumerate(root[0]):
    root_lst.append([x.text, x.tag, x.tail, x.attrib])

music_df = pd.DataFrame(root_lst, columns=["text", "tag", "tail", "attrib"])

In [74]:
music_df

,text,tag,tail,attrib
0,Major Version,key,None,{}
1,1,integer,\n\t,{}
2,Minor Version,key,None,{}
3,1,integer,\n\t,{}
4,Date,key,None,{}
5,2021-09-13T22:38:39Z,date,\n\t,{}
6,Application Version,key,None,{}
7,1.0.6.10,string,\n\t,{}
8,Features,key,None,{}
9,5,integer,\n\t,{}


In [92]:
track_lst = []
for x in root[0][17]:
    track_lst.append([x.text, x.tag])

track_df = pd.DataFrame(track_lst, columns = ['text', 'tag'])

In [93]:
track_df.head()

,text,tag
0,232,key
1,\n\t\t\t,dict
2,235,key
3,\n\t\t\t,dict
4,238,key


In [172]:
song_lst = []
for x in root[0][17][1]: # [1] because we are parsing only the first track
    song_lst.append([x.text, x.tag])

song_df = pd.DataFrame(song_lst)

In [175]:
song_df

,Track ID,key
0,Track ID,key
1,232,integer
2,Name,key
3,Happy Is a Yuppie Word,string
4,Artist,key
5,Switchfoot,string
6,Album Artist,key
7,Switchfoot,string
8,Album,key
9,Nothing Is Sound,string
